In [1]:
import pandas as pd 
from bs4 import BeautifulSoup 
import numpy as np 
import seaborn as sns 
import requests

In [2]:
urls=[]

html=requests.get('https://danskejernbaner.dk/vis.stations.oversigt.php')
html=html.text
soup=BeautifulSoup(html, 'html.parser')

links=soup.find_all("a")

for link in links:
    if "vis.station.php?" in link["href"]:
        urls.append(link.attrs["href"])
df=pd.DataFrame(urls)      
df.columns={"links"}
df

,links
0,vis.station.php?FORLOEB_ID=1326&content=Abildh...
1,vis.station.php?FORLOEB_ID=1331&content=Adelve...
2,vis.station.php?FORLOEB_ID=1539&content=Adsbøl...
3,vis.station.php?FORLOEB_ID=294&content=Agedrup...
4,vis.station.php?FORLOEB_ID=793&content=Agerbæk...
...,...
2171,vis.station.php?FORLOEB_ID=2319&content=Årøsun...
2172,vis.station.php?FORLOEB_ID=2082&content=Aarøsu...
2173,vis.station.php?FORLOEB_ID=719&content=Aasbækb...
2174,vis.station.php?FORLOEB_ID=625&content=Aasen+T...


In [40]:
import time 
unusefull=[]
list_df =list()
new_urls = list()
for link in urls:
    if "æ" or "Æ" in link:
        link = link.replace("æ","%C3%A6").replace("Æ","%C3%A6")
    if "ø" or "Ø" in link:
        link = link.replace("ø","%C3%B8").replace("Ø","%C3%B8")
    if "å" or "Å" in link:
        link = link.replace("å","%C3%A5").replace("Å","%C3%A5")
    new_urls.append(link)

for i in new_urls:
    try:
        df = pd.read_html(f"https://danskejernbaner.dk/{i}", decimal=',')
        list_df.append(df)
    except:
        next  
        
        


In [49]:
list_df[7][0].set_index(0).loc["GPS koordinater"].tolist()[0].split(",")

['55.658089', '12.353550']

In [53]:
final_list=[]


lon = list()
lat = list()

for i in range(len(list_df)):
    try:
        if list_df[i][0].iloc[2][1] == 'I drift':
            final_list.append(i)
    except:
        next
        
for i in range(len(final_list)):
    lon.append(list_df[i][0].set_index(0).loc["GPS koordinater"].tolist()[0].split(",")[0])
    lat.append(list_df[i][0].set_index(0).loc["GPS koordinater"].tolist()[0].split(",")[1])
    
final_df = pd.DataFrame({"lon":lon,"lat":lat})
final_df.head()


,lon,lat
0,55.658215,8.730940
1,55.751927,8.717152
2,54.937063,9.612413
3,55.429704,10.488886
4,55.602028,8.805512


In [54]:
final_df.shape

(480, 2)

In [55]:
final_df.to_csv("stationsdata")

In [59]:
#benytter GEOPY til at beregne afstand: 
#først merge dokumenterne 
from geopy.distance import geodesic

for i, h in #fredes dokument (lon,lat):
    location = (i["lon"], h["lat"])
    df["distance"] = df.apply(lambda final_df:geodesic((final_df["Lon"], final_df["Lat"]), location, axis=1)
    m=min(df["distance"])
    df["distance"] = df.apply(lambda x:geodesic((x["Latitude"], x["Longitude"]), location), axis=1)
    

SyntaxError: invalid syntax (<ipython-input-59-113dcbbffef0>, line 6)

In [70]:
#original som virker 

df = pd.DataFrame({'Latitude':[47.5112, 47.7210, 47.7379, 47.5208, 47.6168],
                    'Longitude':[-122.257, -122.319, -122.233, -122.393, -122.045]})

location = (47.631872, -122.217109)
df["distance"] = df.apply(lambda x:geodesic((x["Latitude"], x["Longitude"]), location), axis=1)

df["distance"]

0    13.748182799073302 km
1    12.519555035527109 km
2    11.848856269639601 km
3    18.100165573313678 km
4    13.044739896967613 km
Name: distance, dtype: object